In [1]:
import pandas as pd 
import numpy as np

In [3]:
faults = pd.read_csv('../data/J1939Faults.csv')
onboard = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

/tmp/ipykernel_13274/31407276.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv('../data/J1939Faults.csv')


In [4]:
onboard.dtypes

Id          int64
Name       object
Value      object
FaultId     int64
dtype: object

In [5]:
faults.dtypes

RecordID                   int64
ESS_Id                     int64
EventTimeStamp            object
eventDescription          object
actionDescription        float64
ecuSoftwareVersion        object
ecuSerialNumber           object
ecuModel                  object
ecuMake                   object
ecuSource                  int64
spn                        int64
fmi                        int64
active                      bool
activeTransitionCount      int64
faultValue               float64
EquipmentID               object
MCTNumber                  int64
Latitude                 float64
Longitude                float64
LocationTimeStamp         object
dtype: object

In [6]:
on_faults = pd.merge(faults, onboard, left_on = 'RecordID', right_on = 'FaultId').drop(columns = ['Id', 'FaultId'])

In [7]:


# Get rid of the rows where the EquipmentID is more than 5 characters
on_faults_filtered = on_faults.loc[on_faults['EquipmentID'].astype(str).str.len() <= 5]

# Filter out faults where the light is going off
on_faults_filtered = on_faults_filtered.loc[on_faults['active'] == True].reset_index(drop = True)



In [8]:
on_faults_filtered['EquipmentID'].nunique()

1841

In [9]:
on_faults_filtered.dtypes

RecordID                   int64
ESS_Id                     int64
EventTimeStamp            object
eventDescription          object
actionDescription        float64
ecuSoftwareVersion        object
ecuSerialNumber           object
ecuModel                  object
ecuMake                   object
ecuSource                  int64
spn                        int64
fmi                        int64
active                      bool
activeTransitionCount      int64
faultValue               float64
EquipmentID               object
MCTNumber                  int64
Latitude                 float64
Longitude                float64
LocationTimeStamp         object
Name                      object
Value                     object
dtype: object

In [10]:
# Convert EventTimeStamp column to datetime object
on_faults_filtered['EventTimeStamp'] = pd.to_datetime(on_faults_filtered['EventTimeStamp'])


In [11]:
# Set EquipmentID and EventTimeStamp columns as index
on_faults_filtered.set_index(['EquipmentID', 'EventTimeStamp', 'spn'], inplace=True)

In [12]:

# Pivot the data into a time series format
full_derates_ts = on_faults_filtered.pivot_table(index=['EquipmentID', 'EventTimeStamp', 'spn'],
                                          values=['RecordID', 'ESS_Id', 'eventDescription', 
                                                 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 
                                                 'ecuMake', 'ecuSource', 'fmi', 'active', 'activeTransitionCount', 
                                                 'faultValue', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp'])

# Reset the index
full_derates_ts.reset_index(inplace=True)

# Sort the data by EquipmentID and EventTimeStamp
full_derates_ts.sort_values(by=['EquipmentID', 'EventTimeStamp'], inplace=True)

# Group the data by EquipmentID
faults_grouped = full_derates_ts.groupby('EquipmentID')

# Create a dictionary of time series, with EquipmentID as the key and the time series as the value
faults_ts_dict = {group: data.drop('EquipmentID', axis=1).set_index('EventTimeStamp').sort_index() 
               for group, data in faults_grouped}


# Fill missing values with 0
for ts in faults_ts_dict.values():
    ts.fillna(0, inplace=True)



/tmp/ipykernel_13274/856689262.py:2: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  full_derates_ts = on_faults_filtered.pivot_table(index=['EquipmentID', 'EventTimeStamp', 'spn'],


Now we have a dictionary where the keys are the specific Equipment Id's and the info inside of them pertians to the types of service codes that came up with a timestamp value. That way we can see if there is info preceding a 5246 "breakdown" code that could predict it coming ahead of time. 

Lets look at one of the trucks with the most data inculuded in its dictionary entry

In [14]:
max_key = max(faults_ts_dict, key=lambda x: faults_ts_dict[x].shape[0])


In [15]:
max_ts = faults_ts_dict.get(max_key)

In [16]:
print(max_ts)

                      spn      ESS_Id   Latitude  Longitude  MCTNumber  \
EventTimeStamp                                                           
2015-04-29 09:50:25   111   2151299.0  32.456111 -91.757824  105436925   
2015-04-29 18:44:14   111   2164374.0  33.321111 -87.046759  105436925   
2015-04-30 04:43:51   111   2168215.0  35.136990 -84.896481  105436925   
2015-04-30 09:25:52   111   2173858.0  35.136157 -84.898425  105436925   
2015-05-03 04:37:47   111   2219673.0  34.662453 -86.752824  105436925   
...                   ...         ...        ...        ...        ...   
2018-09-28 08:10:18   111  63178019.0  36.066342 -86.434305  105439799   
2018-11-28 07:26:59  3216  69584128.0  37.393009 -86.783287  105439799   
2018-11-29 15:24:33   639  69800718.0  39.699351 -77.733611  105439799   
2018-12-03 06:27:42   111  70093771.0  36.580277 -82.317314  105439799   
2019-01-23 10:35:16   639  75095393.0  35.588287 -86.444027  105439799   

                      RecordID  activ